# Dissonancereduction

In [4]:
from adaptivetuning import Dissonancereduction
import numpy as np

Construct an example first, an A major chord in closed position with added octave (A4, C#5, E5, A4) in 12TET.

In [10]:
ji_intervals = [1, 16/15, 9/8, 6/5, 5/4, 4/3, 45/32, 3/2, 8/5, 5/3, 9/5, 15/8, 2]
partials_vol_piano = np.array([3.7, 5.4, 1.2, 1.1, 0.95, 0.6, 0.5, 0.65, 0.001, 0.1, 0.2]) / 5.4

notes = [0, 4, 7, 12]
et_fundamentals = [440 * 2**(i/12) for i in notes]  # equal tempered version of that chord
ji_fundamentals = [440 * ji_intervals[i] for i in notes]  # just version
fundamentals_vol = np.ones(len(notes))
partials_pos = list(range(1, len(partials_vol_piano) + 1))
partials_vol = partials_vol_piano
relative_bounds = (2**(-1/24), 2**(1/24))

The partials of the tonic are used as fixed positions, the rest is to be tuned.

In [11]:
fixed_freq = [et_fundamentals[0] * p for p in partials_pos]
fixed_vol = [fundamentals_vol[0] * v for v in partials_vol]

In [12]:
dissonancereduction = Dissonancereduction()

First factors that we treat as constants during optimization are calculated.

In [13]:
relevant_pairs, critical_bandwidths, volume_factors = dissonancereduction.quasi_constants(
    np.array(et_fundamentals[1:]), np.array(fundamentals_vol[1:]),
    np.array(partials_pos), np.array(partials_vol),
    np.array(fixed_freq), np.array(fixed_vol))

Dropping pairs with volume factor = 0 or distance > 1.46 x critical bandwidth reduces the amount of relevant pairs from 1210 to 140.

In [14]:
len(relevant_pairs)

140

If [i,k,j,l] in relevant_pairs than the ith partial of the kth complex tone and the jth partial of the lth overtone form a relevant pair. If l = -1, j, l refers to the jth fixed frequency instead.

The second overtone of the tonic (2, -1) and the first overtone of the fifths (1, 1) are close -> relevant.

In [15]:
[1, 1, 2, -1] in relevant_pairs.tolist()

True

The fundamental of the tonic (0, -1) and the first overtone of the fifths (1, 1) are not close -> irrelevant.

In [18]:
[1, 1, 0, -1] in relevant_pairs.tolist()

False

The eighth overtones of the third and fifths are close -> relevant.

In [17]:
[0, 8, 1, 8] in relevant_pairs.tolist()

True

But since the eighth overtone of our piano is very week, the volume factor of the pair is small:

In [20]:
volume_factors[relevant_pairs.tolist().index([0, 8, 1, 8])]

0.8608141448259226

The first overtones are strong -> big volume factor.

In [21]:
volume_factors[relevant_pairs.tolist().index([0, 1, 1, 1])]

4.5313189239810825

First overtones of the third and the fifths ([0, 1, 1, 1]) are approximately at 1200 HZ where the critical bandwidth is approximately 200 Hz, our approximation is very rough of course.

In [22]:
critical_bandwidths[relevant_pairs.tolist().index([0, 1, 1, 1])]

187.33314724834622

Third overtones of the third and the fifths ([0, 3, 1, 3]) are approximately at 2400 HZ where the critical bandwidth is approximately 380 Hz.

In [24]:
critical_bandwidths[relevant_pairs.tolist().index([0, 3, 1, 3])]

373.04659900888373

In [25]:
dissonance, gradient = dissonancereduction.dissonance_and_gradient(
    np.array(et_fundamentals[1:]), np.array(partials_pos), np.array(fixed_freq),
    np.array(critical_bandwidths), np.array(volume_factors), np.array(relevant_pairs))

The most dissonant note of an equal tempered major chord is the major third which is to sharp -> the biggest value of the gradient is the one corresponding to the major third and the negative gradient is pointing in the negative direction, corresponding to a down-tuning of the third.

In [27]:
gradient

array([ 0.00593165, -0.00090389, -0.0016534 ])

In [28]:
result = dissonancereduction.tune(np.array(et_fundamentals[1:]), np.array(fundamentals_vol[1:]),
                                  np.array(partials_pos), np.array(partials_vol),
                                  np.array(fixed_freq), np.array(fixed_vol))

In [29]:
result

      fun: 0.8749697723835244
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00059601,  0.00027074, -0.0001622 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 28
      nit: 5
   status: 0
  success: True
        x: array([549.99032273, 659.77210576, 880.40175926])

The resulting chord is more similar to a just major chord than to a equal tempered major chord:

In [33]:
[f / et_fundamentals[0] for f in [et_fundamentals[0]] + result['x'].tolist()]

[1.0, 1.2499780061934875, 1.4994820585387896, 2.0009130892203113]

In [34]:
[f / et_fundamentals[0] for f in et_fundamentals]

[1.0, 1.259921049894873, 1.4983070768766815, 2.0]

In [35]:
[f / ji_fundamentals[0] for f in ji_fundamentals]

[1.0, 1.25, 1.5, 2.0]